In [1]:
# !mkdir -p /home/jovyan/jars
# !wget https://jdbc.postgresql.org/download/postgresql-42.7.4.jar -O /home/jovyan/jars/postgresql-42.7.4.jar
# pip install gspread
# import gspread
# gspread.__version__
import json
cfg = json.load(open('cred.json'))

DBUSER = cfg.get('Connect DB')['DBUSER']
DBPASS = cfg.get('Connect DB')['DBPASS']
DBHOST = cfg.get('Connect DB')['DBHOST']
DBPORT = cfg.get('Connect DB')['DBPORT']
DBNAME = cfg.get('Connect DB')['DBNAME']

import psycopg2
conn = psycopg2.connect(f"host={DBHOST} dbname={DBNAME} user={DBUSER} password={DBPASS} port={DBPORT}")
# conn.close()
cursor = conn.cursor()
sql_stmt = "select table_name from information_schema.tables where table_schema = 'public'"
cursor.execute(sql_stmt)
tables = cursor.fetchall()
print(tables)
cursor.close()
conn.close()

[('pg_stat_statements_info',), ('pg_stat_statements',), ('random_data',), ('random_table',)]


In [3]:
import pyspark
from pyspark.sql import SparkSession
DBUSER = cfg.get('Connect DB')['DBUSER']
DBPASS = cfg.get('Connect DB')['DBPASS']
DBHOST = cfg.get('Connect DB')['DBHOST']
DBPORT = cfg.get('Connect DB')['DBPORT']
DBNAME = cfg.get('Connect DB')['DBNAME']
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkByExamples.com") \
    .config("spark.jars", "/home/jovyan/jars/postgresql-42.7.4.jar") \
    .config("spark.driver.extraClassPath", "/home/jovyan/jars/postgresql-42.7.4.jar") \
    .getOrCreate()

+--------------------+
|          table_name|
+--------------------+
|pg_stat_statement...|
|  pg_stat_statements|
|         random_data|
|        random_table|
+--------------------+



In [4]:
def write_by_jdbc(sdf, db_table_name,  cfg):
    DBUSER = cfg.get('Connect DB')['DBUSER']
    DBPASS = cfg.get('Connect DB')['DBPASS']
    DBHOST = cfg.get('Connect DB')['DBHOST']
    DBPORT = cfg.get('Connect DB')['DBPORT']
    DBNAME = cfg.get('Connect DB')['DBNAME']
    
    sdf.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", "jdbc:postgresql://{DBHOST}:{DBPORT}/{DBNAME}".format(DBHOST = DBHOST, DBPORT=DBPORT, DBNAME=DBNAME)) \
    .option("dbtable", db_table_name) \
    .option("user", DBUSER) \
    .option("password", DBPASS) \
    .option("driver", 'org.postgresql.Driver')\
    .save()

In [6]:
db_table_name = "public.tmp_random_data"
N = 100
# create dataframe random
df = spark.range(N)
sdf = df.selectExpr("id as id")
write_by_jdbc(sdf, db_table_name, cfg)

In [7]:
query_stmt = f"select * from {db_table_name}"
# create dataframe
df = spark.read.format("jdbc") \
    .option("url", f"jdbc:postgresql://{DBHOST}:{DBPORT}/{DBNAME}") \
    .option("query", query_stmt) \
    .option("user", DBUSER) \
    .option("password", DBPASS) \
    .load()
 
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows

